In [ ]:
from pathlib import Path
import sys

from PIL import Image
import open3d as o3d
import torch

sys.path.append('/home/universal/Downloads/dev.sk_robot_rgbd_data/src')
from skrgbd.calibration.calibrations.small_scale_sphere import Calibration
from skrgbd.calibration.trajectories.camera_sphere import CameraCalibrationSphere
from skrgbd.data.data_structure import map_point_id_to_i
from skrgbd.devices.robot.robot_on_sphere import RobotOnSphere
from skrgbd.data.rv_scan import RVScan

sys.path.append('/home/universal/Downloads/dev.sk_robot_rgbd_data/debug/depth_map_reprojection_example/dev.mvs4df/src')
from mvs4df.modules.pointcloud_rendering.render_points import render_points

In [ ]:
scan_dir = '/home/universal/Downloads/dev.sk_robot_rgbd_data/stl_shared_folder/scans/test_calib_board_folder'
scan_i = 0
camera_scans_dir = '/home/universal/Downloads/dev.sk_robot_rgbd_data/experiments/test_calibration/scanning/test_calib_board'
renders_dir = '/home/universal/Downloads/dev.sk_robot_rgbd_data/experiments/test_calibration/renders'

In [ ]:
calibration = Calibration()
trajectory = CameraCalibrationSphere(RobotOnSphere(simulation=True))

In [ ]:
scan = RVScan(scan_dir, scan_i)
scan.load_colors()

In [ ]:
camera = 'phone_left_ir'
# render_scale = 4
Path(f'{renders_dir}/{camera}').mkdir(parents=True, exist_ok=True)

In [ ]:
camera_model = calibration.cam_model[camera]
w, h = camera_model.size_wh

device = 'cuda:0'
camera_model = camera_model.to(device)

In [ ]:
mm_to_meters = torch.zeros(4, 4)
mm_to_meters[3, 3] = 1
mm_to_meters[0, 0] = mm_to_meters[1, 1] = mm_to_meters[2, 2] = 1 / 1000

scan_to_stl_right_at_zero_point = (
    calibration.stl_sphere_extrinsics[scan_i] @ calibration.rv_calib_to_stl_right
    @ mm_to_meters @ scan.board_to_mesh.inverse() @ scan.mesh_to_world.inverse()
)
stl_right_to_camera = calibration.rig_to_cam[camera] @ calibration.rig_to_cam['stl_right'].inverse()

In [ ]:
for pos_i in [0, 11, 12, 23, 24, 35, 36, 47, 48, 58, 59, 69, 70, 80, 81, 90, 91, 99]:
    camera_at_zero_to_camera_at_pos = calibration.cam_sphere_extrinsics[camera][pos_i].inverse()
    matrix = camera_at_zero_to_camera_at_pos @ stl_right_to_camera @ scan_to_stl_right_at_zero_point
    vertices = scan.vertices @ matrix[:3, :3].T + matrix[:3, 3]
    vertices = vertices.T
    
    uv = camera_model.project_fine(vertices.to(device)).cpu()
    torch.cuda.empty_cache()

    render = render_points(
        scan.colors.unsqueeze(0),
        vertices[2].unsqueeze(0),
        uv.unsqueeze(1) * render_scale,
        (h * render_scale, w * render_scale),
        point_radius=(2 ** -.5),
    #     uv_averaging_range=1e-4,
        depth_averaging_range=1e-5,
    ).squeeze(0)
    
    render = render.where(render.isfinite(), render.new_zeros([]))
    render = render.permute(1, 2, 0).clamp(0, 255).byte()
    render = Image.fromarray(render.numpy())
    render.save(f'{renders_dir}/{camera}/{pos_i:04}.png')